In [1]:
import pandas as pd
import pickle
import numpy as np

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
# PATH WHERE THE FILES ARE SAVED
path = './' #r'/dbfs/FileStore/tables/' # databricks filesystem
azure_path = None #'/mnt/ddf/' # Azure Blob Storage

# IMPORT OF DATABASE
ddf = pd.read_excel(path + r'/2_cleaned_data/clean_ddf.xlsx')

# IMPORT OF VARIABLES FROM MODELING NOTEBOOK
modeling = (path + r'/4_outputs/var_results_modeling.sav')
results = pickle.load(open(modeling, 'rb'))

# IMPORT OF DATA FROM FRAMEWORK CONFIGURATION
frameworkconfig1 = (path + r'/3_variables/var1_framework_config.sav')
ddf_config_var = pickle.load(open(frameworkconfig1, 'rb'))

# frameworkconfig2 = (path + r'/3_variables/var2_framework_config.sav')
# ddf_config_kpi = pickle.load(open(frameworkconfig2, 'rb'))

frameworkconfig3 = (path + r'/3_variables/var3_framework_config.sav')
ddf_config_par = pickle.load(open(frameworkconfig3, 'rb'))

In [3]:
# PRIMARY KEY
primary_key = ddf_config_var.loc[np.where(ddf_config_var["VariableUsage"]=='PRIMARY')[0][0],'VariableName']

# FILTER BY COLUMN
filter_by = ddf_config_par.loc[0,'Value']

# FILTER BY VALUE
filter_value = ddf_config_par.loc[1,'Value']

if str(filter_by)!='nan':
    # PRIMARY KEY LIST WITH FILTER 
    pk_list = ddf[primary_key][ddf[filter_by]==filter_value].unique()
else:
    # PRIMARY KEY LIST 
    pk_list = ddf[primary_key].unique()
    
# DATETIME VARIABLE
date_column = ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='DATETIME')[0][0],'VariableName']

# PERIODS TO PREDICT
forecast = pd.to_numeric(ddf_config_par.loc[17,'Value'])

# TARGET VARIABLE
target = ddf_config_var.loc[np.where(ddf_config_var["VariableUsage"]=='TARGET')[0][0],'VariableName']

# EXOGENOUS VARIABLE
exogenous = []
for i in ddf_config_var.loc[np.where(np.logical_and(ddf_config_var["VariableUsage"]=='EXOGENOUS',ddf_config_var["VariableType"]=='NUMERIC'))[0],'VariableName']:
    exogenous.append(i)
for i in ddf_config_var.loc[np.where(np.logical_and(ddf_config_var["VariableUsage"]=='EXOGENOUS',ddf_config_var["VariableType"]=='CATEGORICAL'))[0],'VariableName']:
    exogenous.append(i)

In [ ]:
# FORMAT OF DATETIME
form = ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='DATETIME')[0],'Obs'][0]

# DATETIME FREQUENCY
if ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='DATETIME')[0],'UpdateFrequency'][0]=='days':
    freq = 'D'
elif ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='DATETIME')[0],'UpdateFrequency'][0]=='weeks':
    freq = 'W'
elif ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='DATETIME')[0],'UpdateFrequency'][0]=='months':
    freq = 'M'
elif ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='DATETIME')[0],'UpdateFrequency'][0]=='years':
    freq = 'Y'

# SET OF PRIMARY KEYS WITH COMPLETENESS
ddf_completeness = ddf[[date_column,primary_key]]
ddf_completeness[date_column] = pd.to_datetime(ddf_completeness[date_column], format=form)
pk_completeness = ddf_completeness[[date_column,primary_key]].groupby(primary_key).agg(['min','max','count'])
pk_completeness.columns = ['FirstDate','LastDate','Total']
pk_completeness['ExpectedLength'] = pd.to_datetime(pk_completeness['LastDate'], format=form)-pd.to_datetime(pk_completeness['FirstDate'], format=form)
pk_completeness['ExpectedLength'] = round(pk_completeness['ExpectedLength']/np.timedelta64(1,freq)+1,0)
pk_completeness2 = set(pk_completeness.index[np.where(pk_completeness['Total']==pk_completeness['ExpectedLength'])])
non_completeness = set(pk_list).difference(pk_completeness2)
pk_list = np.array(list(set(pk_list).intersection(pk_completeness2)))
pk_list.sort()

# Sort Values (must) 

In [5]:
ddf.sort_values([primary_key, date_column], inplace=True)

In [ ]:
# FINAL SARIMAX MODEL

parameters_sarimax_opt = list(results[[primary_key,'parameters_sarimax']].drop_duplicates()['parameters_sarimax'])
product_sarimax_opt = list(results[[primary_key,'parameters_sarimax']].drop_duplicates()[primary_key])
mape_sarimax_opt = list(results[[primary_key,'mape_sarimax']].drop_duplicates()['mape_sarimax'])


product_sarimax = []
forecast_sarimax = []
mape_sarimax = []
conf_int_sarimax = []

for pro in pk_list:
    p_params,d_params,q_params,P_params,D_params,Q_params,m_params,t_params = parameters_sarimax_opt[product_sarimax_opt.index(pro)]
    # HISTORICAL DATA
    data = ddf[target][ddf[primary_key]==pro]
    # DATA OF EXOGENOUS VARIABLES
    exog = ddf[exogenous][ddf[primary_key]==pro]
    if exog.shape[1]==0:
        model_sarimax_fit = SARIMAX(np.array(data),order=(p_params,d_params,q_params),
                                    seasonal_order=(P_params,D_params,Q_params,m_params),
                                    trend=t_params,enforce_stationarity=False,enforce_invertibility=False).fit()
        product_sarimax.append(pro)
        forecast_sarimax.append(model_sarimax_fit.get_forecast(steps=forecast).predicted_mean)
        mape_sarimax.append(mape_sarimax_opt[product_sarimax_opt.index(pro)])
        conf_int_sarimax.append(model_sarimax_fit.get_forecast(steps=forecast).conf_int())
    else:
        model_sarimax_fit = SARIMAX(np.array(data),exog=np.array(exog),order=(p_params,d_params,q_params),seasonal_order=(P_params,D_params,Q_params,m_params),trend=t_params,enforce_stationarity=False,enforce_invertibility=False).fit()
        product_sarimax.append(pro)
        exog_post = pd.DataFrame(index=range(forecast))
        for i in range(0,exog.shape[1]):
            if isinstance(exog.loc[exog.index[0],exogenous[i]], float)==True:
                exog_post = pd.concat([exog_post, pd.DataFrame(np.repeat(np.matrix(np.mean(exog.iloc[:,i])),forecast,axis=0))], axis=1)
            else:
                exog_post = pd.concat([exog_post, pd.DataFrame(np.repeat(np.matrix(exog.iloc[:,i].mode()),forecast,axis=0))[0]], axis=1)
        forecast_sarimax.append(model_sarimax_fit.get_forecast(steps=forecast, exog=exog_post).predicted_mean)
        mape_sarimax.append(mape_sarimax_opt[product_sarimax_opt.index(pro)])
        conf_int_sarimax.append(model_sarimax_fit.get_forecast(steps=forecast, exog=exog_post).conf_int())

In [ ]:
# FINAL ETS MODEL
parameters_ets_opt = list(results[[primary_key,'parameters_ets']].drop_duplicates()['parameters_ets'])
product_ets_opt = list(results[[primary_key,'parameters_ets']].drop_duplicates()[primary_key])
mape_ets_opt = list(results[[primary_key,'mape_ets']].drop_duplicates()['mape_ets'])

product_es = []
forecast_es = []
mape_es = []

for pro in pk_list:
    t_params,d_params,s_params,p_params,b_params,r_params = parameters_ets_opt[product_ets_opt.index(pro)]
    # HISTORICAL DATA
    data = ddf[target][ddf[primary_key]==pro]
    model_es_fit = ExponentialSmoothing(np.array(data),trend=t_params,damped=d_params,seasonal=s_params,seasonal_periods=p_params).fit(optimized=True, use_boxcox=b_params, remove_bias=r_params)
    product_es.append(pro)
    forecast_es.append(model_es_fit.forecast(steps=forecast))
    mape_es.append(mape_ets_opt[product_ets_opt.index(pro)])

In [8]:
# DATAFRAME SUMMARY (FORECAST)
predict_sarimax_final = pd.DataFrame(product_sarimax)
predict_sarimax_final.columns = [primary_key]
predict_sarimax_final['predict_sarimax'] = forecast_sarimax
predict_sarimax_final['mape_sarimax'] = mape_sarimax

# predict_sarimax_final = predict_sarimax_final.predict_sarimax.apply(pd.Series)\
# .merge(predict_sarimax_final, left_index = True, right_index = True)\
# .drop (["predict_sarimax"], axis = 1)\
# .melt (id_vars = [primary_key,'mape_sarimax'], value_name = "predict_sarimax")\
# .sort_values(by=[primary_key,'variable'])\
# .dropna()

predict_sarimax_final = pd.melt(predict_sarimax_final.predict_sarimax.apply(pd.Series).merge(predict_sarimax_final, left_index = True, right_index = True)\
.drop (['predict_sarimax'], axis = 1),id_vars=[primary_key,'mape_sarimax'], value_vars=list(range(0,forecast)))\
.sort_values(by=[primary_key,'variable'])\
.dropna()

conf_int = pd.DataFrame()
for i in range(0,len(pk_list)):
  conf_int = conf_int.append(pd.DataFrame(conf_int_sarimax[i]))
conf_int.columns = ['2.5%','97.5%']

predict_sarimax_final = predict_sarimax_final.reset_index(drop=True)
conf_int = conf_int.reset_index(drop=True)

predict_sarimax_final = predict_sarimax_final.join(conf_int)

# DATAFRAME SUMMARY (FORECAST)
predict_es_final = pd.DataFrame(product_es)
predict_es_final.columns = [primary_key]
predict_es_final['predict_es'] = forecast_es
predict_es_final['mape_es'] = mape_es

# predict_es_final = predict_es_final.predict_es.apply(pd.Series)\
# .merge(predict_es_final, left_index = True, right_index = True)\
# .drop (["predict_es"], axis = 1)\
# .melt (id_vars = [primary_key,'mape_es'], value_name = "predict_es")\
# .sort_values(by=[primary_key,'variable'])\
# .dropna()\
# .drop([primary_key], axis = 1)

predict_es_final = pd.melt(predict_es_final.predict_es.apply(pd.Series).merge(predict_es_final, left_index = True, right_index = True).drop (["predict_es"], axis = 1),id_vars=[primary_key,'mape_es'], value_vars=list(range(0,forecast))).sort_values(by=[primary_key,'variable']).dropna()

# JOIN OF DATAFRAMES
merged_results = pd.merge(predict_sarimax_final, predict_es_final,  how='left', left_on=[primary_key,'variable'], right_on = [primary_key,'variable'])
merged_results = merged_results.rename(columns={'value_x': 'sarimax_forecast', 'value_y': 'ets_forecast'})

In [9]:
# EXCEL WITH THE DATA OF MODELS
writer = pd.ExcelWriter(path + r'/4_outputs/ddf_forecast.xlsx')
merged_results.to_excel(writer,'Sheet1')
writer.save()

In [10]:
if azure_path is not None:
    dbutils.fs.cp(path.replace('/dbfs/','dbfs:/') + "/ddf_forecast.xlsx", azure_path + '/4_outputs/ddf_forecast.xlsx')